In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

Selectors :
   * percentagePerAlternative : checkboxe (T/F) value to display the bar chart in percentage or not
   * xaxisAlternativeResAlt : dropdown value indicating the number of bar per chart (Alls, 5, 25, 50, 10 first, 10 last)
   * orderAlternativeResAlt : dropdown value indicating how to order xlabel : alphabetically or by y value (hight to low or low to hight)
   * orderLabelValuesAlternativeResAlt : dropdown value indicating how to order labels : by the sum of the 2 superposed values or by true or false response

In [ ]:
#constructing a dropdown to select how many data we want to see in visualisations for x axis
xaxisAlternativeResAlt=widgets.Dropdown(options=[('All data',0),('5 per visualisation',5),('25 per visualisation',25),('50 per visualisation',50),('10 first',-1),('10 last',-2)],
                            description='x axe',
                            disabled=False)

#constructing a dropdown to order for alphabetic keys or for values
orderAlternativeResAlt=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                            description='Order',
                            disabled=False)

#constructing a checkboxe to view chart per percentage per alternative
percentageAlternativeResAlt=widgets.Checkbox(value=False,
                                        description='Show % per alternative')

#constructing a dropdown value indicating how to order labels : by the sum of the 2 superposed values or by true or false response
orderLabelValuesAlternativeResAlt=widgets.Dropdown(options=[("Order by all true and false responses values",0),
                                                ("Order by true responses values","true"),
                                                ("Order by false responses values","false")],
                                        description='Change order label',
                                        disabled=False)

***vis_correct_incorrect_question* function**:
* Output : the widgets selector and the outputResPerAlt in a list to be display in a VBox
* outputResPerAlt : a bar chart showing the total number of correct and incorrect alternatives selected by players in each multiple-choice question

In [ ]:
def vis_correct_incorrect_question():
    def display_correct_incorrect_question(change):
        if orderAlternativeResAlt.value==0 or percentageAlternativeResAlt.value:
            orderLabelValuesAlternativeResAlt.layout=layout_hidden
        else:
            orderLabelValuesAlternativeResAlt.layout=layout_visible
        with outputResPerAlt:
            clear_output(wait=False)
            correct_incorrect_question()
            clear_output(wait=True)
    xaxisAlternativeResAlt.observe(display_correct_incorrect_question, 'value')
    orderAlternativeResAlt.observe(display_correct_incorrect_question, 'value')
    percentageAlternativeResAlt.observe(display_correct_incorrect_question, 'value')
    orderLabelValuesAlternativeResAlt.observe(display_correct_incorrect_question, 'value')
    display_correct_incorrect_question(None)
    return [percentageAlternativeResAlt,
            xaxisAlternativeResAlt,
            orderAlternativeResAlt,
            orderLabelValuesAlternativeResAlt,
            outputResPerAltUpdate,
            outputResPerAlt]

***correct_incorrect_question* function**:

Output :  Displays bar chart showing for each user the number of correct and incorrect alternatives selected in multiple-choice questions

In [ ]:
def correct_incorrect_question():
    #get all players and alternatives selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    if len(alternatives_list_selected)>0 and len(playersSelected)>0:
        if xaxisAlternativeResAlt.value in [0,-1,-2]:
            nbgraph=1
        else:
            div=len(alternatives_list_selected)/xaxisAlternativeResAlt.value
            nbgraph=math.ceil(div)
        accordionResPerQuestion=widgets.Accordion(children=[widgets.Output() for i in range(nbgraph)],
                                         selected_index=None)
        if nbgraph>1:
            for i in range(nbgraph):
                accordionResPerQuestion.set_title(i, str(i+1)+"/"+str(nbgraph))
        display(accordionResPerQuestion)
        def on_selected_index_ResPerQuestion_change(change):
            n=change['new']
            if n!=None:
                #close and clear previous fig of alternatives
                for fig in figAltQuestion:
                    plt.close(fig=fig)
                figAltQuestion.clear()
                #get all players and alternatives selected
                playersSelected=checkboxesPlayersSelected.get_selected_options()
                alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
                # alternatives correct and incorrect responses
                true_false_per_alternative = {}
                alt_max=0
                if len(alternatives_list_selected)>0:
                    for alt in alternatives_list_selected:
                        true_false_per_alternative[alt] = {}
                        alt_true = 0
                        alt_false = 0
                        for player in playersSelected:
                            if alt in players_info[player]["alternatives"].keys():
                                _, results = zip(*players_info[player]["alternatives"][alt])
                                alt_true += results.count("True") +results.count(True)
                                alt_false += results.count("False") +results.count(False)
                                if alt_true+alt_false>alt_max:
                                    alt_max=alt_true+alt_false
                        true_false_per_alternative[alt]["true"]=alt_true
                        true_false_per_alternative[alt]["false"]=alt_false
                    alt_max+=0.5
                #define order of labels values
                if orderLabelValuesAlternativeResAlt.value==0:
                    lambdafunc=lambda item:item[1]["true"]+item[1]["false"]
                else:
                    lambdafunc=lambda item:item[1][orderLabelValuesAlternativeResAlt.value]
                if percentageAlternativeResAlt.value:
                    lambdafunc=lambda item:item[1]["true"]/(item[1]["true"]+item[1]["false"])
                #order alls values
                true_false_per_alternative = orderDict(true_false_per_alternative,orderAlternativeResAlt.value,lambdafunc)
                #separe in sub dict in  function of xaxis dropout selected
                nbgraphshown, true_false_per_alternative,N = organizeDictPerxAxisSelected(playersSelected,xaxisAlternativeResAlt.value, true_false_per_alternative)
                with accordionResPerQuestion.children[n]:
                    #get value for true and false answer for this graph
                    true_per_alternative=[true_false_per_alternative[n][alt]["true"] for alt in true_false_per_alternative[n].keys()]
                    false_per_alternative=[true_false_per_alternative[n][alt]["false"] for alt in true_false_per_alternative[n].keys()]
                    list_alt=[alt for alt in true_false_per_alternative[n].keys()]
                    if n<nbgraphshown-1:
                        ind = np.arange(N)
                    else:
                        ind = np.arange(len(list_alt))
                    #create new fig with custom name in the tab
                    name=filename+ "_correct_incorrect_per_question"
                    if nbgraphshown>1:
                        name+="_"+str(n+1)+"_"+str(nbgraphshown)
                    fig = plt.figure(num=name, constrained_layout=True)
                    fig.canvas.header_visible = False
                    fig.canvas.layout.min_height = '400px'
                    figAltQuestion.append(fig)
                    #display graph
                    if percentageAlternativeResAlt.value:
                        # Bar chart showing the total percentage of incorrect alternatives
                        # selected by players in each multiple-choice question
                        get_percentage_bar(true_per_alternative,false_per_alternative,list_alt)
                        ylabel="Percentage of good responses"
                        title="Percentage of good responses per alternative"
                    else:
                        # Bar chart showing the total number of correct and incorrect alternatives
                        # selected by players in each multiple-choice question
                        get_two_superposed_values_bars(ind,list_alt,
                                       true_per_alternative, "Correct",'g',
                                       false_per_alternative, "Incorrect",'r')
                        ylabel="Number of responses"
                        title="Number of correct and incorrect responses per alternative"
                    #add axes labels and title
                    plt.xlabel("Alternative")
                    plt.ylabel(ylabel)
                    plt.title(title)
                    #orient xticks
                    plt.xticks(rotation=90)
                    plt.show()
        accordionResPerQuestion.observe(on_selected_index_ResPerQuestion_change, 'selected_index')
        accordionResPerQuestion.selected_index=0
    else:
        #create new fig with custom name in the tab
        name=filename+ "_correct_incorrect_per_question"
        fig = plt.figure(num=name, constrained_layout=True)
        fig.canvas.header_visible = False
        fig.canvas.layout.min_height = '400px'
        figAltQuestion.append(fig)
        #display no data to fill vis
        noDataToFillVis(10)
        if percentageAlternativeResAlt.value:
            ylabel="Percentage of good responses"
            title="Percentage of good responses per alternative"
        else:
            ylabel="Number of responses"
            title="Number of correct and incorrect responses per alternative"
        #add axes labels and title
        plt.xlabel("Alternative")
        plt.ylabel(ylabel)
        plt.title(title)
        plt.show()
    with outputResPerAltUpdate:
        clear_output(wait=False)